In [ ]:
# 1. Environment Setup & Data Loading

from IPython.display import display
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os

# Configuration Settings
warnings.filterwarnings('ignore')          
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_rows', 100)     
plt.style.use('seaborn-v0_8-whitegrid')    

# Define file paths
BASE_PATH = "../../data"
MAIN_FILE = "corn_climate_risk_futures_daily_master.csv"
SHARE_FILE = "corn_regional_market_share.csv"

# Load Data
print("Loading data...")

df = pd.read_csv(f"{BASE_PATH}/{MAIN_FILE}")
market_share = pd.read_csv(f"{BASE_PATH}/{SHARE_FILE}")
print("Data loaded successfully.")

# Display Data Overview
print("\n" + "="*50)
print(f"Main Dataset Shape: {df.shape}")
print("="*50)
display(df.head())

# Important: Identification of target rows (This will help prevent ID mismatch)
target_col = 'futures_close_ZC_1'
if target_col in df.columns:
    missing_targets = df[target_col].isnull().sum()
    print(f"\nTarget column: {target_col}")
    print(f"Rows with missing targets (to be predicted): {missing_targets}")

Loading data...
Data loaded successfully.

Main Dataset Shape: (320661, 41)


,ID,crop_name,country_name,country_code,region_name,region_id,harvest_period,growing_season_year,date_on,climate_risk_cnt_locations_heat_stress_risk_low,climate_risk_cnt_locations_heat_stress_risk_medium,climate_risk_cnt_locations_heat_stress_risk_high,climate_risk_cnt_locations_unseasonably_cold_risk_low,climate_risk_cnt_locations_unseasonably_cold_risk_medium,climate_risk_cnt_locations_unseasonably_cold_risk_high,climate_risk_cnt_locations_excess_precip_risk_low,climate_risk_cnt_locations_excess_precip_risk_medium,climate_risk_cnt_locations_excess_precip_risk_high,climate_risk_cnt_locations_drought_risk_low,climate_risk_cnt_locations_drought_risk_medium,climate_risk_cnt_locations_drought_risk_high,futures_close_ZC_1,futures_close_ZC_2,futures_close_ZW_1,futures_close_ZS_1,futures_zc1_ret_pct,futures_zc1_ret_log,futures_zc_term_spread,futures_zc_term_ratio,futures_zc1_ma_20,futures_zc1_ma_60,futures_zc1_ma_120,futures_zc1_vol_20,futures_zc1_vol_60,futures_zw_zc_spread,futures_zc_zw_ratio,futures_zs_zc_spread,futures_zc_zs_ratio,date_on_year,date_on_month,date_on_year_month
0,8af42722-3f05-4ede-80fc-605e0e2b3b67,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2017,2016-06-15,23,0,0,23,0,0,23,0,0,16,7,0,429.00,434.00,477.50,1156.0,-0.017182,-0.017331,5.00,1.011655,414.4125,387.695833,375.014583,0.013520,0.015724,48.50,0.898429,727.00,0.371107,2016,6,2016_06
1,54f4ddc5-e7ab-4bfb-ad6a-5649841af563,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2017,2016-06-16,23,0,0,23,0,0,23,0,0,14,9,0,425.25,430.50,472.50,1134.5,-0.008741,-0.008780,5.25,1.012346,415.7000,388.616667,375.512500,0.013799,0.015792,47.25,0.900000,709.25,0.374835,2016,6,2016_06
2,63a41fce-d371-4295-a58a-dc6491664020,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2017,2016-06-17,23,0,0,14,1,8,23,0,0,14,8,1,437.75,442.75,481.25,1159.5,0.029394,0.028971,5.00,1.011422,418.0875,389.770833,376.122917,0.013442,0.016145,43.50,0.909610,721.75,0.377533,2016,6,2016_06
3,cddfa440-e0eb-4735-beb1-1aca2afefe53,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2017,2016-06-18,23,0,0,20,0,3,23,0,0,15,6,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,2016_06
4,3eaacfe1-29be-4da9-b5c9-a9457d2d2b83,Corn: Commodity Tracked,Argentina,AR,Buenos Aires,bffad37a-7c60-432f-984a-8ea83a944311,Harvest,2017,2016-06-19,23,0,0,21,2,0,23,0,0,16,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,6,2016_06



Target column: futures_close_ZC_1
Rows with missing targets (to be predicted): 101130


In [3]:
# Convert 'date_on' to datetime objects
df['date_on'] = pd.to_datetime(df['date_on'])

# Merge the main dataframe with market share data on 'region_id'
# Using 'left' join is correct to preserve all rows from the main climate dataset
df_merged = pd.merge(df, market_share[['region_id', 'percent_country_production']], 
                     on='region_id', 
                     how='left')

# Check for rows where market share might be missing after merge
missing_share = df_merged['percent_country_production'].isnull().sum()
print(f"Rows with missing market share: {missing_share}")

# Fill missing market share with 0 (assuming negligible production if not listed)
df_merged['percent_country_production'] = df_merged['percent_country_production'].fillna(0)

# Sort data by Date and Region ID for proper time-series structure
# We keep the 'ID' column intact for the final submission mapping
df_merged = df_merged.sort_values(by=['date_on', 'region_id']).reset_index(drop=True)

# CRITICAL CHECK: Ensure row count matches the original dataset
# If row counts don't match, the submission will fail the ID check
print("\n" + "="*50)
print(f"Original Row Count: {df.shape[0]}")
print(f"Merged Row Count:   {df_merged.shape[0]}")
print("="*50)

if df.shape[0] != df_merged.shape[0]:
    print("WARNING: Row count changed! This will cause ID mismatch error later.")

# Display the updated dataframe structure
display(df_merged[['date_on', 'region_name', 'percent_country_production']].head())

print("\n")
df_merged.info()

Rows with missing market share: 3635

Original Row Count: 320661
Merged Row Count:   320661


,date_on,region_name,percent_country_production
0,2016-01-01,Republic of Mordovia,1.0
1,2016-01-01,Vinnytsia,10.0
2,2016-01-01,Bryansk,2.0
3,2016-01-01,Paraná,11.0
4,2016-01-01,Henan,8.0




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320661 entries, 0 to 320660
Data columns (total 42 columns):
 #   Column                                                    Non-Null Count   Dtype         
---  ------                                                    --------------   -----         
 0   ID                                                        320661 non-null  object        
 1   crop_name                                                 320661 non-null  object        
 2   country_name                                              320661 non-null  object        
 3   country_code                                              320661 non-null  object        
 4   region_name                                               320661 non-null  object        
 5   region_id                                                 320661 non-null  object        
 6   harvest_period                                            320661 non-null  object        
 7   growing_season_year        